In [184]:
"""
# Used only for Dev

import simulator, agent, simulator_interface, behavior
from importlib import reload
reload(simulator)
reload(behavior)
reload(simulator_interface)
reload(agent)

"""

<module 'agent' from '/Users/jalalizekki/ia/simple-playgrounds-epuck/client/agent.py'>

In [185]:
from simulator_interface import open_session, close_session
simulator, epuck1, epuck2 = open_session(nb_agents=2)

In [186]:
epuck1.wait(3)

epuck1.left_spd = 3
epuck1.right_spd = 3

epuck1.wait(3)

epuck1.left_spd = 0
epuck1.right_spd = 0

In [187]:
close_session(simulator)